In [2]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

def get_category_colour(category):
    
    if category == "normal":
        return "#17becf"
    elif category == "glucose":
        return "#2ca02c"
    elif category == "athlete":
        return "#9467bd"
    elif category == "protein":
        return "#e377c2"
        
def build_size_list_and_hover_text_per_category(category,barcode,scores):
    
    sub_scores = scores[scores.category == category]
    found_sample_index_match = sub_scores["sample_id"].str.contains("^"+barcode+"$").values
    size_list = []
    hover_text = []
    default_point_size = 10
    selected_point_size = 20
    ns,nc = sub_scores.shape
    i = 0
    while (i < ns):
        if found_sample_index_match[i] == True:
            size_list.append(selected_point_size)
        else:
            size_list.append(default_point_size)
        hover_text.append("Sample: "+sub_scores.iloc[i].sample_id) 
        i = i + 1
    return size_list,hover_text

def plot_scores_per_category(fig,category,barcode,scores):
    
    size_list, hover_text = build_size_list_and_hover_text_per_category(category,barcode,scores)
    fig.add_trace(go.Scatter(
                        x=scores[scores.category == category].pc1.values,
                        y=scores[scores.category == category].pc2.values,
                        name=category,
                        mode='markers',
                        marker=dict(size=size_list,color=get_category_colour(category)),
                        hovertext=hover_text,
                        hoverinfo="all",
                   ))
    
    return fig

def plot_scores(barcode,scores):
 
    fig = go.Figure()
    fig = plot_scores_per_category(fig,"normal",barcode,scores)
    fig = plot_scores_per_category(fig,"glucose",barcode,scores)
    fig = plot_scores_per_category(fig,"athlete",barcode,scores)
    fig = plot_scores_per_category(fig,"protein",barcode,scores)
    fig.update_layout(title="PCA scores plot of PC1 and PC2",
                       xaxis_title="PC1",
                       yaxis_title="PC2",
                       showlegend=True)
    fig.update_xaxes(range=[-1, 1])
    fig.update_yaxes(range=[-1, 1])
    fig.show()
    
def plot_spectra(barcode,scores,spectra):
    
    category = scores[scores.sample_id == barcode].category.values[0]
    fig = go.Figure()
    fig.add_trace(go.Scatter(
                        x=spectra.index.values,
                        y=spectra[barcode],
                        name=barcode,
                        mode='lines',
                        line=dict(color=get_category_colour(category))
                     ))
    fig.add_trace(go.Scatter(
                    x=spectra.index.values,
                    y=spectra['NPC-00005231'],
                    name="normal",
                    mode='lines',
                    line=dict(color=get_category_colour('normal'))
                 ))
    fig.update_xaxes(range=[10,-0.2])
    fig.update_layout(title="NMR Spectrum for sample "+barcode,
                      xaxis_title="ppm",
                       yaxis_title="Intensity")
    fig.show()
    
def identify_sample(barcode,scores,spectra):
    
    barcode = barcode.strip()

    if barcode not in ["NPC-00005231","NPC-00005232","NPC-00005233","NPC-00005234"]:
        print("Sample not found")

    plot_scores(barcode,scores)
    plot_spectra(barcode,scores,spectra)
    
spectra = pd.read_csv("spectra.csv").T
new_header = spectra.iloc[0] #grab the first row for the header
spectra = spectra[1:] #take the data less the header row
spectra.columns = new_header
scores = pd.read_csv('scores.csv')

In [5]:
# Please scan your barcode here and re-run this cell.

barcode = """NPC-00005234

"""
identify_sample(barcode,scores,spectra)